In [1]:
import os
import sys
from collections import OrderedDict

In [2]:
sys.path.append("../..")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import spatialpy

In [5]:
import matplotlib.pyplot as plt
import numpy

# Global Constants
MAX_X_DIM = 5.0
MIN_X_DIM = -5.0
TOL = 1e-9

In [6]:
class Edge1(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0] - MAX_X_DIM) < 0.05
class Edge2(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0] - MIN_X_DIM) < 0.05


In [7]:
class cylinderDemo3D(spatialpy.Model):
    def __init__(self, model_name="cylinder_demo3d"):
        spatialpy.Model.__init__(self, model_name)

        # System constants
        D_const = 0.1

        # Define Species
        A = spatialpy.Species(name="A", diffusion_constant=D_const)
        B = spatialpy.Species(name="B", diffusion_constant=D_const)
        self.add_species([A, B])

        # Define Geometry
        self.mesh = spatialpy.Mesh.read_xml_mesh('cylinder.xml')

        # Define Subdomains
        self.add_subdomain(Edge1(), 2)
        self.add_subdomain(Edge2(), 3)
        
        # Restrict the movement of Chemical Species
        self.restrict(A,[1,2])
        self.restrict(B,[1,3])


        k_react = spatialpy.Parameter(name="k_react", expression=1.0)
        k_creat1 = spatialpy.Parameter(name="k_creat1", expression=100)
        k_creat2 = spatialpy.Parameter(name="k_creat2", expression=100)
        self.add_parameter([k_react, k_creat1,k_creat2])


        # Define Reactions
        R1 = spatialpy.Reaction(reactants=None, products={A:1}, 
                                rate=k_creat1, restrict_to=2)
        R2 = spatialpy.Reaction(reactants=None, products={B:1}, 
                              rate=k_creat2, restrict_to=3)
        R3 = spatialpy.Reaction(reactants={A:1, B:1}, products=None, 
                              rate=k_react)
        self.add_reaction([R1, R2, R3])

        # Define simulation timespan
        self.set_timesteps(1, 200)

In [8]:
model = cylinderDemo3D()

In [9]:
from spatialpy.nsmsolver import NSMSolver

In [10]:
#result = model.run(report_level=2)
sol = NSMSolver(model, report_level=2)
%time sol.compile()

Compiling Solver.  Build dir: /tmp/tmpelyl3322
Creating propensity file /tmp/tmpelyl3322/cylinder_demo3d_generated_model.c
find_h = 0.8703111740074692
cmd: cd /tmp/tmpelyl3322 ; make -f /home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/build/Makefile.nsm ROOT=/home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine MODEL=/tmp/tmpelyl3322/cylinder_demo3d_generated_model.c BUILD=/tmp/tmpelyl3322

gcc -c -o linked_list.o /home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/src/linked_list.c -I/home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/include/ -O3 -Wall 
gcc -c -o particle.o /home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/src/particle.c -I/home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/include/ -O3 -Wall 
gcc -c -o simulate.o /home/brian/Desktop/research/SpatialPy/spatialpy/ssa_sdpd-c-simulation-engine/src/simulate.c -I/ho

In [11]:
os.listdir(sol.build_dir)

['main.o',
 'count_cores.o',
 'particle.o',
 'ssa_sdpd',
 'simulate_rdme.o',
 'linked_list.o',
 'binheap.o',
 'simulate.o',
 'output.o',
 'cylinder_demo3d_generated_model.c']

In [12]:
%time result = sol.run()

cmd: cd /tmp/tmpsgqce83m;/tmp/tmpelyl3322/ssa_sdpd

CPU times: user 0 ns, sys: 7.47 ms, total: 7.47 ms
Wall time: 10.4 s


In [13]:
os.listdir(result.result_dir)

['output165.vtk',
 'output29.vtk',
 'output41.vtk',
 'output167.vtk',
 'output32.vtk',
 'output143.vtk',
 'output35.vtk',
 'diffusion_matrix_1560731950',
 'output126.vtk',
 'output199.vtk',
 'output8.vtk',
 'output21.vtk',
 'output13.vtk',
 'output61.vtk',
 'output173.vtk',
 'output27.vtk',
 'output12.vtk',
 'output109.vtk',
 'output107.vtk',
 'output5.vtk',
 'output50.vtk',
 'output157.vtk',
 'output85.vtk',
 'output121.vtk',
 'output145.vtk',
 'output159.vtk',
 'output19.vtk',
 'output33.vtk',
 'output72.vtk',
 'output96.vtk',
 'output101.vtk',
 'output179.vtk',
 'output125.vtk',
 'output45.vtk',
 'output188.vtk',
 'output200.vtk',
 'output144.vtk',
 'output37.vtk',
 'output92.vtk',
 'output68.vtk',
 'output87.vtk',
 'output4.vtk',
 'output134.vtk',
 'output16.vtk',
 'output138.vtk',
 'output17.vtk',
 'output168.vtk',
 'output66.vtk',
 'output140.vtk',
 'output132.vtk',
 'output80.vtk',
 'output123.vtk',
 'output47.vtk',
 'output44.vtk',
 'output147.vtk',
 'output52.vtk',
 'output7.v

In [14]:
len(os.listdir(result.result_dir))

203